In [2]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")


Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [26]:
from docx import Document
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

# Load the Word document
doc = Document(r"C:\Users\013224\Downloads\chatgpt.docx")

text = ''
tables_data = []

# Extract text and table data
for element in doc.element.body:
    if element.tag.endswith('tbl'):  # If it's a table
        table = [
            [' '.join(paragraph.text if paragraph.text is not None else '' for paragraph in cell)]
            for row in element
            for cell in row
        ]
        tables_data.append(table)
    elif element.text is not None:  # For non-table elements
        text += element.text

# Load model & tokenizer
model_name = "deepset/roberta-base-squad2"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Question-answering pipeline using the extracted text as the context
nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)
question = "what are the years having revenue more than 130 crore?"

# Combining text and table data for context
context = '\n\n'.join([
    text,
    *['\n'.join(['\t'.join(cell) for cell in table]) for table in tables_data]
])

res = nlp(question=question, context=context)

answer = res['answer']
confidence = res['score']  # Confidence level of the answer

# Define a confidence threshold
confidence_threshold = 0.00

# Check if the answer is not present in the context or if the confidence is low
if confidence < confidence_threshold or answer not in context or not answer.strip():
    print("I don't know.")
else:
    print(f"Answer: {answer}")
    print(f"Confidence Level: {confidence}")


Answer: 2017
Confidence Level: 0.017780479043722153
